In [8]:
import sys, os

PROJECT_ROOT = "/home/nlin/workspace/code/projects/autograd_cpp"
sys.path.append(os.path.join(PROJECT_ROOT, "build"))

import torch
import hypothesis
import math
import lamppp

In [ ]:
a = lamppp.cVariable(lamppp.cTensor([1, 2, 3, 4, 5], [1, 5]), True)
b = lamppp.cVariable(lamppp.cTensor([1, 2, 3, 4, 5], [1, 5]), True)

c = lamppp.add(a, b)
c.backward()
print(c)